# Inicializace

In [47]:
from collections import deque               # Ukladani stavu
from __future__ import division             # Deleni realnych cisel (kvuli verzi Pythonu 2.6)
import sys                                  # Pro navazani knihovny Open AI Gym
import numpy as np                          # Knihovna pro operace s maticemi
sys.path.append('/home/xbucha02/libraries') # Adresa knihovny Open AI Gym
import gym                                  # Knihovna Open AI Gym
from gym import wrappers                    # Pomucka na nahravani
env = gym.make('CartPole-v0')            # Konkretni hra z Open AI Gym
env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu
gpuMemoryUsage=1                            # Vyuziti pameti GPU
import tensorflow as tf                     # Knihovna TensorFlow pro sdileni GPU
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
set_session(tf.Session(config=config))
import random

# Parametry
observetime = 500                  # Pocet framu kazde hry
startEpsilon = 1                   # Pravdepodobnost konani nahodneho tahu na zacatku
endEpsilon = 0.1                   # Pravdepodobnost konani nahodneho tahu na konci
currentEpsilon = startEpsilon      # Soucasna pravdepodobnost konani nahodneho tahu
epsilonDiminution = 0.9995          # Hodnota snizovani epsilonu
gamma = 0.95                       # Snizujici faktor
minibatchSize = 32                 # Velikost minibatche
episodes = 5000                    # Pocet epizod
#numberOfGames = 20                # Pocet her na konci
actionCount = env.action_space.n   # Pocet akci v danem prostredi
stateSize = env.observation_space.shape[0]
learningRate = 0.001             # Learning rate
memorySize = 2000                  # Replay pamet

# Model NN

In [48]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

def getDQN(actionCount,stateSize):
    net = Sequential()
    #net.add(Dense(20, activation="relu", kernel_initializer="uniform", input_shape=(2, 2)))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu", input_dim=stateSize))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu"))
    #net.add(Dropout(rate=0.5))
    #net.add(Dense(8, activation="relu", kernel_initializer="uniform"))
    
    net.add(Dense(actionCount, activation="linear"))
    
    net.summary()
    net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=learningRate), metrics=['accuracy'])
    
    return net

In [49]:
net = getDQN(actionCount, stateSize)
observation = env.reset()                    # Reset prostredi
memory = deque(maxlen=memorySize)

for eps in range (episodes):  
    #stateF = state[0][1] #np.reshape(state, [1, stateSize])
    #D = deque() #vyprazdneni D
    state = env.reset()
    env.render()
    #obs = np.expand_dims(observation, axis=0) 
    #state = np.stack((obs, obs), axis=1)

    i = 0
    for t in range(observetime):
        #if i == 0:
            
        #else:
        #    if i==4:
        #        i = 0
        #    else:
        #        i = i + 1
        if np.random.rand() <= currentEpsilon:
            action = np.random.randint(0, actionCount, size=1)[0]
        else:
            #print("shirt")
            Q = net.predict(state)
            #print("shirt")
            action = np.argmax(Q)
            #print("{}".format(action))
        #print ("state: {}" .format(state))
        #print("{}".format(action))
        
        #state = np.reshape(state[0][1], [1, stateSize])
        #print ("state: {}" .format(state))
        observation_new, reward, done, info = env.step(int(action))
        
        #print()
        #stateNewF = observation_new
        #print("heje{}".format(stateNewF))
        #obs_new = np.expand_dims(observation_new, axis=0)        
        #print ("obs_new: {}" .format(obs_new))
        #state_new = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)
        #state = state[0][0]#next_state = np.reshape(next_state, [1, stateSize])
        next_state = np.reshape(observation_new, [1, stateSize])
        #next_state = [[next_state]]
        memory.append((state, action, reward, next_state, done))
        #print(state, action, reward, stateNewF, done)
        print ("state: {} state_new: {}" .format(state, next_state))
        
        state = next_state         # Update state
        #print ("state: {}" .format(state))
        #print ("__________________________________________")
        if done:
            print("Episode: {}/{}, score: {}".format(eps, episodes, t))
            break
    if eps > 100:
        minibatch = random.sample(memory, minibatchSize) #z D vybere pocet mb_size samplu

        #inputs_shape = (minibatchSize,) + state.shape[1:]
        #print("inputs_shape: {}".format(inputs_shape))
        #inputs = np.zeros(inputs_shape)
        #print("inputs: {}".format(inputs))
        #targets = np.zeros((minibatchSize, env.action_space.n)) #vytvori pole ([], []), 1. argument - radky, 2. argument - sloupce
        #print("targets: {}".format(targets))

        for i in range(0, minibatchSize):
            print("{}" .format(minibatch))
            stateMb = minibatch[i][0]
            actionMb = minibatch[i][1]
            rewardMb = minibatch[i][2]
            state_newMb = minibatch[i][3]
            doneMb = minibatch[i][4]

            #target = reward

            # Build Bellman equation for the Q function
            #inputs[i:i+1] = np.expand_dims(state, axis=0)

            #print("inputs: {}".format(inputs))
            #targets[i] = net.predict(state)
            #print("targets: {}".format(targets))
            #print("{}".format(state_newMb))
            Q_sa = net.predict(state_newMb)
            print("{}, {}".format(state_newMb, Q_sa))
            if done:
                targetMb = rewardMb
            else:
                targetMb = reward + gamma * np.max(Q_sa)
            
            
            target_f = net.predict(stateMb)
            
            target_f[0][actionMb] = targetMb

            #print("inputs: {}, targets:{}" .format(inputs, targets))

            net.fit(stateMb, target_f, epochs=1, verbose=0)

        #net.train_on_batch(inputs, targets)

        if currentEpsilon > endEpsilon:
            currentEpsilon = currentEpsilon * epsilonDiminution
        print("{}".format(currentEpsilon))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_50 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_51 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
state: [-0.00106572 -0.0233702   0.01819544 -0.02572084] state_new: [[-0.00153312  0.17148615  0.01768102 -0.31260782]]
state: [[-0.00153312  0.17148615  0.01768102 -0.31260782]] state_new: [[ 0.0018966   0.36635181  0.01142887 -0.59966268]]
state: [[ 0.0018966   0.36635181  0.01142887 -0.59966268]] state_new: [[  9.22363497e-03   5.61312014e-01  -5.64385560e-04  -8.88723870e-01]]
state: [[  9.22363497e-0

state: [ 0.01204016 -0.04072764  0.00334546  0.01672235] state_new: [[ 0.01122561  0.15434618  0.00367991 -0.27490317]]
state: [[ 0.01122561  0.15434618  0.00367991 -0.27490317]] state_new: [[ 0.01431254  0.34941543 -0.00181815 -0.56642318]]
state: [[ 0.01431254  0.34941543 -0.00181815 -0.56642318]] state_new: [[ 0.02130084  0.54456284 -0.01314662 -0.85967835]]
state: [[ 0.02130084  0.54456284 -0.01314662 -0.85967835]] state_new: [[ 0.0321921   0.3496224  -0.03034018 -0.57115794]]
state: [[ 0.0321921   0.3496224  -0.03034018 -0.57115794]] state_new: [[ 0.03918455  0.15493876 -0.04176334 -0.28818553]]
state: [[ 0.03918455  0.15493876 -0.04176334 -0.28818553]] state_new: [[ 0.04228332 -0.03956352 -0.04752705 -0.00896137]]
state: [[ 0.04228332 -0.03956352 -0.04752705 -0.00896137]] state_new: [[ 0.04149205  0.15620668 -0.04770628 -0.31625276]]
state: [[ 0.04149205  0.15620668 -0.04770628 -0.31625276]] state_new: [[ 0.04461619 -0.03820444 -0.05403134 -0.0389882 ]]
state: [[ 0.04461619 -0.03

state: [[ 0.02913555 -0.14990958  0.00501576  0.33672278]] state_new: [[ 0.02613736  0.04514063  0.01175022  0.04562577]]
state: [[ 0.02613736  0.04514063  0.01175022  0.04562577]] state_new: [[ 0.02704017 -0.15014782  0.01266274  0.34199271]]
state: [[ 0.02704017 -0.15014782  0.01266274  0.34199271]] state_new: [[ 0.02403722  0.0447917   0.01950259  0.05332961]]
state: [[ 0.02403722  0.0447917   0.01950259  0.05332961]] state_new: [[ 0.02493305  0.23962866  0.02056918 -0.23313685]]
state: [[ 0.02493305  0.23962866  0.02056918 -0.23313685]] state_new: [[ 0.02972562  0.43445076  0.01590644 -0.51926127]]
state: [[ 0.02972562  0.43445076  0.01590644 -0.51926127]] state_new: [[ 0.03841464  0.62934521  0.00552122 -0.80688966]]
state: [[ 0.03841464  0.62934521  0.00552122 -0.80688966]] state_new: [[ 0.05100154  0.82439105 -0.01061657 -1.09783073]]
state: [[ 0.05100154  0.82439105 -0.01061657 -1.09783073]] state_new: [[ 0.06748936  0.62941047 -0.03257319 -0.80849756]]
state: [[ 0.06748936  0.

state: [[ 0.04469365  0.41935132 -0.0471215  -0.6947694 ]] state_new: [[ 0.05308067  0.6150941  -0.06101689 -1.00190625]]
state: [[ 0.05308067  0.6150941  -0.06101689 -1.00190625]] state_new: [[ 0.06538256  0.42083818 -0.08105501 -0.72899247]]
state: [[ 0.06538256  0.42083818 -0.08105501 -0.72899247]] state_new: [[ 0.07379932  0.61698143 -0.09563486 -1.04604548]]
state: [[ 0.07379932  0.61698143 -0.09563486 -1.04604548]] state_new: [[ 0.08613895  0.42325    -0.11655577 -0.78485002]]
state: [[ 0.08613895  0.42325    -0.11655577 -0.78485002]] state_new: [[ 0.09460395  0.22990596 -0.13225277 -0.53099157]]
state: [[ 0.09460395  0.22990596 -0.13225277 -0.53099157]] state_new: [[ 0.09920207  0.42661601 -0.1428726  -0.86224901]]
state: [[ 0.09920207  0.42661601 -0.1428726  -0.86224901]] state_new: [[ 0.10773439  0.2336982  -0.16011758 -0.61768253]]
state: [[ 0.10773439  0.2336982  -0.16011758 -0.61768253]] state_new: [[ 0.11240835  0.43065176 -0.17247123 -0.95620755]]
state: [[ 0.11240835  0.

state: [[ 0.02670376 -0.15579751  0.06620732  0.38261631]] state_new: [[ 0.02358781 -0.35179398  0.07385964  0.69541764]]
state: [[ 0.02358781 -0.35179398  0.07385964  0.69541764]] state_new: [[ 0.01655193 -0.54785841  0.087768    1.01040745]]
state: [[ 0.01655193 -0.54785841  0.087768    1.01040745]] state_new: [[ 0.00559476 -0.74403503  0.10797615  1.32931039]]
state: [[ 0.00559476 -0.74403503  0.10797615  1.32931039]] state_new: [[-0.00928594 -0.55042834  0.13456235  1.07227497]]
state: [[-0.00928594 -0.55042834  0.13456235  1.07227497]] state_new: [[-0.02029451 -0.35731682  0.15600785  0.8246683 ]]
state: [[-0.02029451 -0.35731682  0.15600785  0.8246683 ]] state_new: [[-0.02744084 -0.55418917  0.17250122  1.16207091]]
state: [[-0.02744084 -0.55418917  0.17250122  1.16207091]] state_new: [[-0.03852463 -0.36168122  0.19574264  0.92805886]]
state: [[-0.03852463 -0.36168122  0.19574264  0.92805886]] state_new: [[-0.04575825 -0.55883013  0.21430381  1.27531648]]
Episode: 31/5000, score:

...

state: [[ 0.02484583 -0.22720751  0.03208086  0.32543317]] state_new: [[ 0.02030168 -0.42277119  0.03858952  0.62805792]]
state: [[ 0.02030168 -0.42277119  0.03858952  0.62805792]] state_new: [[ 0.01184625 -0.61840987  0.05115068  0.93263996]]
state: [[ 0.01184625 -0.61840987  0.05115068  0.93263996]] state_new: [[ -5.21944849e-04  -8.14183286e-01   6.98034780e-02   1.24094774e+00]]
state: [[ -5.21944849e-04  -8.14183286e-01   6.98034780e-02   1.24094774e+00]] state_new: [[-0.01680561 -0.62002349  0.09462243  0.97092287]]
state: [[-0.01680561 -0.62002349  0.09462243  0.97092287]] state_new: [[-0.02920608 -0.8162793   0.11404089  1.29176719]]
state: [[-0.02920608 -0.8162793   0.11404089  1.29176719]] state_new: [[-0.04553167 -1.01265131  0.13987623  1.61786726]]
state: [[-0.04553167 -1.01265131  0.13987623  1.61786726]] state_new: [[-0.06578469 -0.81942811  0.17223358  1.37185285]]
state: [[-0.06578469 -0.81942811  0.17223358  1.37185285]] state_new: [[-0.08217325 -1.01623394  0.1996706

[[-0.02184585  0.16385281 -0.02463011 -0.35971646]], [[-0.03842297 -0.03868851]]
[(array([[-0.0212139 , -0.03159718, -0.02343546, -0.05973258]]), 1, 1.0, array([[-0.02184585,  0.16385281, -0.02463011, -0.35971646]]), False), (array([[-0.01259035,  0.57110697, -0.01176665, -0.9305387 ]]), 1, 1.0, array([[ -1.16821122e-03,   7.66385734e-01,  -3.03774270e-02,
         -1.22689589e+00]]), False), (array([[-0.04616764,  0.13113602,  0.10768714,  0.06396887]]), 0, 1.0, array([[-0.04354492, -0.0653519 ,  0.10896652,  0.38859234]]), False), (array([[-0.08605014, -0.38754139,  0.17114094,  0.90139417]]), 0, 1.0, array([[-0.09380096, -0.58451707,  0.18916883,  1.24261147]]), False), (array([[-0.02425343,  0.16231169, -0.03475132, -0.24683531]]), 0, 1.0, array([[-0.02100719, -0.03229715, -0.03968803,  0.03468688]]), False), (array([[ 0.05351273,  0.04230705, -0.03822359,  0.03798986]]), 0, 1.0, array([[ 0.05435887, -0.15224651, -0.03746379,  0.31837205]]), False), (array([[ 0.00434882,  0.3942822

[(array([[-0.0212139 , -0.03159718, -0.02343546, -0.05973258]]), 1, 1.0, array([[-0.02184585,  0.16385281, -0.02463011, -0.35971646]]), False), (array([[-0.01259035,  0.57110697, -0.01176665, -0.9305387 ]]), 1, 1.0, array([[ -1.16821122e-03,   7.66385734e-01,  -3.03774270e-02,
         -1.22689589e+00]]), False), (array([[-0.04616764,  0.13113602,  0.10768714,  0.06396887]]), 0, 1.0, array([[-0.04354492, -0.0653519 ,  0.10896652,  0.38859234]]), False), (array([[-0.08605014, -0.38754139,  0.17114094,  0.90139417]]), 0, 1.0, array([[-0.09380096, -0.58451707,  0.18916883,  1.24261147]]), False), (array([[-0.02425343,  0.16231169, -0.03475132, -0.24683531]]), 0, 1.0, array([[-0.02100719, -0.03229715, -0.03968803,  0.03468688]]), False), (array([[ 0.05351273,  0.04230705, -0.03822359,  0.03798986]]), 0, 1.0, array([[ 0.05435887, -0.15224651, -0.03746379,  0.31837205]]), False), (array([[ 0.00434882,  0.39428224, -0.00831069, -0.62928415]]), 1, 1.0, array([[ 0.01223446,  0.58951918, -0.0208

[[ 0.01950164  0.05334896 -0.08777927 -0.15372744]], [[ 0.0423229   0.06557925]]
[(array([[-0.0212139 , -0.03159718, -0.02343546, -0.05973258]]), 1, 1.0, array([[-0.02184585,  0.16385281, -0.02463011, -0.35971646]]), False), (array([[-0.01259035,  0.57110697, -0.01176665, -0.9305387 ]]), 1, 1.0, array([[ -1.16821122e-03,   7.66385734e-01,  -3.03774270e-02,
         -1.22689589e+00]]), False), (array([[-0.04616764,  0.13113602,  0.10768714,  0.06396887]]), 0, 1.0, array([[-0.04354492, -0.0653519 ,  0.10896652,  0.38859234]]), False), (array([[-0.08605014, -0.38754139,  0.17114094,  0.90139417]]), 0, 1.0, array([[-0.09380096, -0.58451707,  0.18916883,  1.24261147]]), False), (array([[-0.02425343,  0.16231169, -0.03475132, -0.24683531]]), 0, 1.0, array([[-0.02100719, -0.03229715, -0.03968803,  0.03468688]]), False), (array([[ 0.05351273,  0.04230705, -0.03822359,  0.03798986]]), 0, 1.0, array([[ 0.05435887, -0.15224651, -0.03746379,  0.31837205]]), False), (array([[ 0.00434882,  0.3942822

ValueError: Error when checking : expected dense_49_input to have shape (None, 4) but got array with shape (4, 1)